In [1]:
import bokeh as bk
import bokeh.models as models
import bokeh.plotting
from bokeh.plotting import figure, show
import pandas as pd
import polars as pl

In [2]:
df = pd.DataFrame()
df['x'] = [1,2,3,4,5]
df['y'] = [5,4,3,2,1]

In [3]:
source = models.sources.ColumnDataSource(df)

In [4]:
fig = figure()
fig.scatter(source=source)

GlyphRenderer(id='p1038', ...)

In [29]:
show(fig)

In [7]:
pl.DataFrame(df)

x,y
i64,i64
1,5
2,4
3,3
4,2
5,1


In [44]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Legend

# Sample data
source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[6, 7, 2, 4, 5],
    x2=[1, 2, 3, 4, 5],
    y2=[5, 6, 3, 2, 4]
))

# Create a new plot
p = figure()

# Add glyphs with a legend label
line = p.line('x', 'y', source=source, legend_label="Line", line_width=2)
circle = p.circle('x2', 'y2', source=source, size=20, color="navy", legend_label="Circle")

# Manually create a legend (optional, if more control is needed)
legend = Legend(items=[
    ("Line", [line]),
    ("Circle", [circle])
], location="center")

# Clicking the legend's items will hide/mute the glyph
p.legend.click_policy = "hide"  # Can also be "mute"

# Add the legend manually if created (optional)
# p.add_layout(legend)

# Show the result
show(p)


In [11]:
import os
from bokeh.plotting import figure, show
from bokeh.io.export import get_svg, export_svg
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF
from reportlab.graphics.shapes import Drawing
from figs._fig import Fig
import plotly.io as pio
import xml.etree.ElementTree as ETree
import svgpathtools as svgtools

class BokehScalableFigure:
    def __init__(
            self,
            figure: figure,
            plot_width=17, 
            plot_height=11, 
            x_scale=0.25, 
            y_scale=0.25, 
            x_start=None,
            y_start=None
    ):
        # Basic plot setup
        self.plot_width = plot_width * 72
        self.plot_height = plot_height * 72
        self.x_scale = x_scale  # real-world units per inch
        self.y_scale = y_scale  # real-world units per inch
        self.x_start = x_start
        self.y_start = y_start

        self.figure = figure(
            width=self.plot_width, height=self.plot_height,
            output_backend="svg"
        )

        # Default file paths
        self.svg_path = 'temp_plot.svg'
        self.pdf_path = 'temp_plot.pdf'

    def add_line(self, x, y, **kwargs):
        self.figure.line(x, y, **kwargs)

    def write_svg(self, filename=None):
        filename = filename or self.svg_path
        # Save the current figure
        export_svg(self.figure, filename=filename)
    
    @property   
    def root(self):
        root = ETree.parse(self.svg_path).getroot()
        self.root = root
        return self.root
        
    def grid_dimensions(self):
        root = self.root
        elements = [element for element in root.iter()]
        #  get the bounding box of the grid of the svg plot exported from Bokeh
        bbox = svgtools.parse_path(elements[5].attrib['d']).bbox()
        x_length_grid = bbox[1] - bbox[0]
        y_height_grid = bbox[3] - bbox[2]
        
        

    def write_scaled_pdf(self, svg_path=None, pdf_path=None):
        svg_path = svg_path or self.svg_path
        pdf_path = pdf_path or self.pdf_path
        
        # Convert SVG to PDF while maintaining scale
        drawing = svg2rlg(svg_path)
        
        # Adjust the scale based on real-world measurement per inch (dpi assumed to be 72 for SVG)
        scale_x = 1 / self.x_scale
        scale_y = 1 / self.y_scale
        drawing.width = drawing.width * scale_x
        drawing.height = drawing.height * scale_y
        drawing.scale(scale_x, scale_y)

        # Create a ReportLab drawing and render the SVG drawing into it
        rendered_drawing = Drawing(drawing.width, drawing.height)
        rendered_drawing.add(drawing)

        # Write to a PDF
        renderPDF.drawToFile(rendered_drawing, pdf_path)
        print(f'PDF exported to {pdf_path}')
    

# Usage
if __name__ == "__main__":
    bokeh_fig = BokehScalableFigure()
    bokeh_fig.add_line(x=[1, 2, 3, 4], y=[4, 3, 5, 2], line_width=2, color="blue")
    bokeh_fig.write_svg()  # Save the SVG
    bokeh_fig.write_scaled_pdf()  # Convert and scale to PDF


AttributeError: property 'root' of 'BokehScalableFigure' object has no setter

In [66]:
root = ETree.parse('temp_plot.svg').getroot()
elements = [element for element in root.iter()]
#  get the bounding box of the grid of the svg plot exported from Bokeh
bbox = svgtools.parse_path(elements[5].attrib['d']).bbox()
x_min, y_min = bbox[0], bbox[2]
x_length_grid_in_inches = (bbox[1] - bbox[0]) / 72
y_height_grid_in_inches = (bbox[3] - bbox[2]) / 72

In [67]:
x_length_grid_in_inches

16.125

5.5